In [1]:
import requests
from bs4 import BeautifulSoup
import csv
import time


base_site_url = 'https://eg.hatla2ee.com/ar/car/toyota'

pages_to_scrape = 50

output_csv = 'car_info.csv'

retry_limit = 5
delay_factor = 1


car_entries = []
current_pg = 1


while current_pg <= pages_to_scrape:
    constructed_pg_url = f"{base_site_url}/page/{current_pg}"

    attempt = 0
    page_html_content = None
    while attempt < retry_limit:
        try:
            page_response = requests.get(constructed_pg_url)
            page_response.raise_for_status()
            page_html_content = BeautifulSoup(page_response.content, 'html.parser')
            break
        except requests.exceptions.RequestException as error:
            if attempt < retry_limit - 1:
                time.sleep(delay_factor * (2 ** attempt))
                attempt += 1
            else:
                print(f"Unable to retrieve {constructed_pg_url}: {error}")

    if not page_html_content:
        current_pg += 1
        continue

    car_blocks = page_html_content.select('div.newCarListUnit_wrap')
    for car_block in car_blocks:
        car_title = car_block.select_one('div.newCarListUnit_header a').get_text(strip=True)
        car_model = car_block.select('div.newCarListUnit_metaTags span.newCarListUnit_metaLink')[1].get_text(strip=True)
        car_paint = car_block.select_one('span.newCarListUnit_metaTag.mob_hidden').get_text(strip=True)
        car_distance = ""
        meta_data_tags = car_block.select('span.newCarListUnit_metaTag')
        for meta_tag in meta_data_tags:
            if "كم" in meta_tag.get_text():
                car_distance = meta_tag.get_text(strip=True)
                break
        car_location = car_block.select('div.newCarListUnit_metaTags span.newCarListUnit_metaLink')[-1].get_text(strip=True)
        car_date = car_block.select_one('div.otherData_Date span').get_text(strip=True)
        car_cost = car_block.select_one('div.main_price a').get_text(strip=True)

        car_entries.append({
            'title': car_title,
            'car_model_name': car_model,
            'car_color': car_paint,
            'car_mileage': car_distance,
            'location': car_location,
            'date': car_date,
            'price': car_cost
        })

    if len(car_entries) >= 500:
        break
    current_pg += 1

cars_data = car_entries[:500]


with open(output_csv, mode='w', newline='', encoding='utf-8-sig') as csvfile:
    csv_writer = csv.DictWriter(csvfile, fieldnames=['title', 'car_model_name', 'car_color', 'car_mileage', 'location', 'date', 'price'])
    csv_writer.writeheader()
    csv_writer.writerows(cars_data)

print(f'Data has been successfully written to {output_csv}')


Data has been successfully written to car_info.csv
